In [ ]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import imageio

In [ ]:
def plotSpatial(data, fileNumber):
  # Plot spatial representation
  # Assigns green to healthy, orange to at risk, and red to contaminated
  cmap = colors.ListedColormap(['green', 'orange', 'red'])
  plt.figure(figsize=(7, 6))
  plt.pcolor(data, cmap=cmap, edgecolors='k', linewidths=1, vmin=0, vmax=2)
  cbar = plt.colorbar(label="", orientation="vertical", ticks=[0.33, 1, 1.66])
  cbar.ax.set_yticklabels(['healthy', 'at risk', 'contaminated'])
  plt.savefig('figure_' + str(fileNumber) + '.jpg', bbox_inches='tight', pad_inches=0.02)
  plt.close()
  plt.show()
def plotDynamics(data):
  # Plots line graph of the changes of cells over a period of time
  fig, axes = plt.subplots(figsize=(7, 6))
  axes.plot(data[0], data[1], label='healthy', color='green')
  axes.plot(data[0], data[2], label='at_risk', color='orange')
  axes.plot(data[0], data[3], label='contaminated', color='red')
  axes.set_xlabel('Time (months)')
  axes.set_ylabel('Cubic feet of soil')
  axes.legend(bbox_to_anchor=(.3, 1), fontsize=13, fancybox=False, shadow=False, frameon=False)
  plt.show()
  plt.savefig('SoilLeadContamination.png', bbox_inches='tight', pad_inches=0.02)
  plt.close()

In [ ]:
def update_status(domain, time):
  new_domain = np.copy(domain)

  for row in range(1, 61): #iterates through the row
    for col in range(1, 61): #iterates through the column
      #checks to see if element is contaminated, and then checks if neighboring cells are healthy
      # if healthy, they are converted to at risk
        if (domain[row, col] == 2):
          if(domain[row, col+1] == 0):
            new_domain[row, col+1] = 1
          if(domain[row+1, col] == 0):
            new_domain[row+1, col] = 1
          if(domain[row-1, col] == 0):
            new_domain[row-1, col] = 1
          if(domain[row, col-1] == 0):
            new_domain[row, col-1] = 1

        #checks to see if element is at-risk and checks the value of neighboring cells is contaminated
        # if contaminated, count variable is increased for each contaminated cell
        elif (domain[row, col] == 1):
          count = 0
          if(domain[row, col+1] == 2):
            count = count + 1
          if(domain[row+1, col] == 2):
            count = count + 1
          if(domain[row-1, col] == 2):
            count = count + 1
          if(domain[row, col-1] == 2):
            count = count + 1
          if random.random() < count/4:
            new_domain[row, col] = 2
          #if cell is healthy, their is a  10% chance of it be randomly being contaminated from other factors
          elif domain[row, col] == 0:
            x = random.randint(1, 10)
            if x == 1:
              new_domain[row, col] = 2


  if time % 6 == 0:
    # Introduce random changes to the domain
    rows, cols = domain.shape
    #Creates a random row
    random_row = random.randint(0, rows - 1)
    #Creates a random column
    random_col = random.randint (0, cols-1)
    # creates a random location using the previously generated row and column
    random_location = [random_row, random_col]
    # creates a list that will store randomly generated cells
    location_cells = []
    #Iterates through the rows and columns of the randomly generated location
    for r in range(0, random_row):
      for c in range(0, random_col):
        #generates new rows and column indexes
        new_row = random_location[0] + r
        new_col = random_location[1] + c
         #checks to see if the new row and column is greater than 0 and within the boundaries of the domain array
        if 0 <= new_row < rows and 0 <= new_col < cols:
          #adds the new row and column indexes to the location cells
          location_cells.append((new_row, new_col))
    # Randomly chooses cells from the random location
    change_to_healthy = random.sample(location_cells, len(location_cells))
    #Iterates through the random sample and changes all the cells to healthy
    for cell in change_to_healthy:
      #Changes all the cells within the random sample to healthy
      new_domain[cell] = 0
  #returns the newly updated domain
  return new_domain

In [ ]:
def main():
  random.seed(time.time())
  #Initializes a size for the domain array
  sizeX, sizeY = 62,62
  probabilities = [0.9, 0.05, 0.05]
  domain = np.random.choice([0, 1, 2], size=(sizeY, sizeX), p=probabilities)
  #Sets the border cells to different values
  domain[:, 0] = 3
  domain[:, -1] = 3
  domain[0, :] = 3
  domain[-1, :] = 3
  simTime, border, healthy, at_risk, contaminated = [],[],[],[],[]
  currTime = 0
  plotSpatial(domain, currTime)
  simTime.append(currTime)
  border.append(np.count_nonzero(domain==3))
  healthy.append(np.count_nonzero(domain == 0))
  at_risk.append(np.count_nonzero(domain == 1))
  contaminated.append(np.count_nonzero(domain == 2))
  images = []


  for currTime in range(1,101):
    domain = update_status(domain, currTime)
    #Splices out the border values, and runs the plotspatial function 100 times
    plotSpatial(domain[1:61,1:61], currTime)
    simTime.append(currTime) #adds the current time to a list
    # adds the elements with the value of 3 (border) to the list
    border.append(np.count_nonzero(domain == 3))
    # adds  the elements of domain that are healthy (0) to the healthy list
    healthy.append(np.count_nonzero(domain == 0))
    # adds the elements of domain that are at risk (1) to the at risk list
    at_risk.append(np.count_nonzero(domain == 1))
    # adds the elements of domain that are contaiminated (2) to the contaminated list
    contaminated.append(np.count_nonzero(domain == 2))
    #Saves each iteration of the spatial into a seperate jpg file
    filename = 'figure_' + str(currTime) + '.jpg'
    images.append(imageio.v2.imread(filename))
  #Creates the gif
  imageio.mimsave('movie.gif', images, fps =3)
  temporal_dynamics = [simTime, healthy, at_risk, contaminated]
  #Plots the graph
  plotDynamics(temporal_dynamics)

main()